### Generate Archive URLs based on pattern

In [65]:
# Import required libraries
import datetime
import pandas as pd 

# Set the base URL
base_url = 'https://economictimes.indiatimes.com/archive/year-{},month-{}.cms'

# Get the current year and month
current_year = datetime.date.today().year
current_month = datetime.date.today().month

# Create a list of year and month
year_month_list = []
for year in range(2001, current_year+1):
    for month in range(1, 13):
        if year == current_year and month > current_month:
            break
        year_month_list.append(base_url.format(year, month))

df = pd.DataFrame(year_month_list) 
df.to_csv("../data/urls_to_scrape.csv", header=False,index=False)

### Gather Archive links of each date using the previously generated links

In [66]:
import time 
 
from selenium import webdriver 
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

# start by defining the options 
options = webdriver.ChromeOptions() 
options.headless = True # it's more scalable to work in headless mode 
# normally, selenium waits for all resources to download 
# we don't need it as the page also populated with the running javascript code. 
options.page_load_strategy = 'none' 
# this returns the path web driver downloaded 
chrome_path = ChromeDriverManager().install() 
chrome_service = Service(chrome_path) 
# pass the defined options and service objects to initialize the web driver 
driver = Chrome(options=options, service=chrome_service) 
driver.implicitly_wait(5)

daily_link = []

# Scrape news articles from a website
def scrape_news(url):
    driver.get(url)
    content=driver.find_element(By.ID, "calender").find_elements(By.TAG_NAME,"a")
    for a in content:
        url="https://economictimes.indiatimes.com"+a.get_dom_attribute("href")
        daily_link.append(url)

df=pd.read_csv("../data/urls_to_scrape.csv",header=None)
for url in df.values:
    scrape_news(url[0])


dfs = pd.DataFrame(daily_link) 
dfs.to_csv("../data/daily_links.csv", header=False,index=False)



C:\Users\vishn\AppData\Local\Temp\ipykernel_19136\2894986945.py:13: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True # it's more scalable to work in headless mode


### Get individual news links for detailed content gathering

In [ ]:
from selenium import webdriver 
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

options = webdriver.ChromeOptions() 
options.headless = True
options.page_load_strategy = 'none'
chrome_path = ChromeDriverManager().install() 
chrome_service = Service(chrome_path) 
driver = Chrome(options=options, service=chrome_service) 
driver.implicitly_wait(1)
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()


individual_news_links_list = []

# Scrape news articles from a website
def scrape_news(url):
    driver.get(url)
    
    content=driver.find_elements(By.CLASS_NAME, "contentbox5")
    if(content):
        content.pop(0)
        for table in content:
            individual_news=table.find_elements(By.TAG_NAME,"li")
            
            if(individual_news):
                # print("news count "+str(individual_news.count))
                for each_news_li in individual_news:
                    individual_news_url=each_news_li.find_element(By.TAG_NAME,"a").get_attribute("href")
                    # print(individual_news_url)
                    individual_news_links_list.append(individual_news_url)
                    # print(url)

df=pd.read_csv("../data/daily_links.csv",header=None)
length=df[0].count()
progress=0
printProgressBar(progress, length, prefix = 'Progress:', suffix = 'Complete', length = 50)
for url in df.values:
    progress=progress+1
    printProgressBar(progress, length, prefix = 'Progress:', suffix = 'Complete', length = 50)
    # print(url[0])
    scrape_news(url[0])


dfs = pd.DataFrame(individual_news_links_list) 
dfs.to_csv("../data/individual_news_links.csv", header=False,index=False)

### Individual News Scraping

In [8]:
from selenium import webdriver 
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

options = webdriver.ChromeOptions() 
options.headless = True
options.page_load_strategy = 'none'
chrome_path = ChromeDriverManager().install() 
chrome_service = Service(chrome_path)
driver = Chrome(options=options, service=chrome_service)
driver.implicitly_wait(2)

daily_link = []

def multiple_drivers(count):
    driver_list=[]
    for _ in range(count):
        driver = Chrome(options=options, service=chrome_service)
        driver.implicitly_wait(2)
        driver_list.insert(driver)
    return driver_list

def replace_multiline(text):
    return text.replace("\n\n","\n")
# Scrape news articles from a website
def scrape_news(url):
    driver.get(url)
    
    content=driver.find_element(By.TAG_NAME, "article")
    if(content):
        text=content.find_element(By.CLASS_NAME,"medium").text
        text=replace_multiline(text)
        print(text)

df=pd.read_csv("../data/individual_news_links_100.csv",header=None)

def parallel_data(data_from,count):
    return_array = []
    i = data_from
    while i < data_from+count:
        if(df.values[i]):
            return_array.append(df.values[i][0])
            # print(df.values[i][0])
        i += 1
    return return_array

i = 0
c=df[0].count()
count=5
multiple_drivers(count)
while i < c:
    #parallel download
    download=parallel_data(i,count)
    for _ in multiple_drivers:
        scrape_news(download[0])
    # print(download[0])
    scrape_news(download[1])
    # scrape_news(download[2])
    # scrape_news(download[3])
    # scrape_news(download[4])
    i += 5

# for url in df.values:
#     print(url[0])
#     scrape_news(url[0])

C:\Users\vishn\AppData\Local\Temp\ipykernel_3284\4276475783.py:10: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


The developed world, especially the United States, may not like this but in 2008 Japan would be sending into space the world’s first Greenhouse Gases Observing Satellite (GOSAT) which would not only specify the quantum of emission by every region but also observe and analyse the concentration of greenhouse gases in all continents.
The GOSAT data will make it possible to ascertain the global distribution of carbon dioxide and methane and the geographical distribution of all seasonal and inter-annual variation in the flux of greenhouse gases.
The analysis will not only contribute to a deeper scientific understanding of the behaviour of causative global warming agents, but also provide fundamental information to refine climate change predictions and, thus, formulate counter measures. The study will also help in environmental administration efforts such as ascertaining the amount of CO2 absorbed or released in every region as also evaluate the carbon balance in global forests.
The project 

KeyboardInterrupt: 

### Sentimental analysis Model train/test

In [86]:
#Loading the Dataset
import pandas as pd
data = pd.read_csv('../data/financial_sentimental_data.csv')
#Pre-Prcoessing and Bag of Word Vectorization using Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(data['Sentence'])

# print(text_counts.vocabulary_)
# print(text_counts)
#Splitting the data into trainig and testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, data['Sentiment'], test_size=0.25, random_state=5)

# Training the model
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
MNB.fit(X_train, Y_train)
# #Caluclating the accuracy score of the model
from sklearn import metrics
# print(X_test[1])
new_text = ["The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform"]
N_test=cv.transform(new_text)
# print(N_test)
print("")
# print(X_train[0])
predicted = MNB.predict(N_test)
# print(cv.vocabulary_)

print(predicted)
# accuracy_score = metrics.accuracy_score(predicted, Y_test)
# print("Accuracuy Score: ",accuracy_score)
# # text_counts1 = cv.fit_transform()
# X_new=[["This stock is going down in the last 2 months rapidly"],["This stock is going +0.03% every day for the last 2 weeks"]]
# ynew = MNB.predict(X_new)
# # print(text_counts1)
# # predicted1=MNB.predict(X_new[0])

# print(ynew)


['neutral']


c:\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
